Задание 1 (python):
В архиве 5 файлов: 
1. movies.xlsx: id фильма, название фильма и его популярность
2. financials.xlsx: id фильма, его расходы и доходы
3. movie_to_genres.xlsx: id фильма и его жанры
4. sequels.xlsx: сиквелы фильмов (по id)
5. crews.xlsx: кто работал над созданием фильма

Необходимо с помощью jupyter notebook и библиотеки pandas ответить на следующие вопросы по датасету:
1. Выведи самые высокобюджетные жанры в среднем и медианном соотношении. 
А жанры, которые зарабатывают больше всего? Выведи топ 3

2. Выясни, в среднем сиквелы зарабатывают больше своих предшественников?
Выведи топ 3 сиквела, которые в сравнении с предшественником заработали больше денег

3. Создай датасет кинорежиссеров (Director), которые работали на других ролях в кино. (Результат вывода: кинорежиссер, профессия, фильм)
Какие кинорежиссеры самые разносторонние?

In [29]:
import pandas as pd

movies = pd.read_excel('Movies datasets/movies.xlsx')
financials = pd.read_excel('Movies datasets/financials.xlsx')
movie_to_genres = pd.read_excel('Movies datasets/movie_to_genres.xlsx')
sequels = pd.read_excel('Movies datasets/sequels.xlsx')
crews = pd.read_excel('Movies datasets/crews.xlsx')

In [30]:
# 1. Найдем самые высокобюджетные жанры в среднем и медианном соотношении
movies_financials = pd.merge(movies, financials, on='id')
movies_financials_genres = pd.merge(movies_financials, movie_to_genres, left_on='id', right_on='movie_id')

budget_stats_by_genre = movies_financials_genres.groupby('genre')['budget'].agg(['mean', 'median']).reset_index().round()  

budget_stats_by_genre_sorted_mean = budget_stats_by_genre.sort_values(by='mean', ascending=False)

print("Самые высокобюджетные жанры")
print(budget_stats_by_genre_sorted_mean)

Самые высокобюджетные жанры
              genre        mean      median
2         Animation  80826708.0  75000000.0
1         Adventure  76892389.0  60000000.0
8           Fantasy  75640923.0  60000000.0
7            Family  68241749.0  56000000.0
15  Science Fiction  62814049.0  44000000.0
0            Action  62390552.0  45000000.0
17              War  40984556.0  28500000.0
16         Thriller  40953931.0  30000000.0
13          Mystery  38765784.0  30000000.0
10          History  37754868.0  25000000.0
3            Comedy  36155455.0  25000000.0
18          Western  35481642.0  18000000.0
4             Crime  34754720.0  26000000.0
6             Drama  30024848.0  20000000.0
14          Romance  28298663.0  20000000.0
12            Music  23969798.0  17500000.0
11           Horror  20557630.0  13500000.0
5       Documentary   5422902.0   2276824.0
9           Foreign   2645001.0   1000000.0


In [31]:
# Три самых зарабатывающих жанра
revenue_stats_by_genre = movies_financials_genres.groupby('genre')['revenue'].sum().reset_index()    

revenue_stats_by_genre_sorted = revenue_stats_by_genre.sort_values(by='revenue', ascending=False)

top_3_genres_by_revenue = revenue_stats_by_genre_sorted.head(3)
print("Топ-3 жанра по выручке") 
print(top_3_genres_by_revenue)

Топ-3 жанра по выручке
       genre       revenue
1  Adventure  163667924328
0     Action  162316088236
3     Comedy  120412094932


In [33]:
# 2. Сколько сиквелы зарабатывают относительно предшествующих фильмов
financials = pd.read_excel('Movies datasets/financials.xlsx')

sequels_financials = pd.merge(sequels, financials[['id', 'revenue']], left_on='id', right_on='id', how='left')
sequels_financials = pd.merge(sequels_financials, financials[['id', 'revenue']], left_on='sequel', right_on='id', suffixes=('_original', '_sequel'))

sequels_financials['revenue_diff'] = sequels_financials['revenue_sequel'] - sequels_financials['revenue_original']

average_revenue_diff = sequels_financials['revenue_diff'].mean()
print(f"Средняя разница в доходах между сиквелами и их предшественниками: {average_revenue_diff.round()} ")

Средняя разница в доходах между сиквелами и их предшественниками: 151187973.0 


In [34]:
# Топ-3 сиквела, которые заработали больше предшественников
top_3_sequels = sequels_financials.sort_values(by='revenue_diff', ascending=False).head(3)

print("Топ-3 сиквела, которые заработали больше предшественников:")
print(top_3_sequels[['title', 'revenue_original', 'revenue_sequel', 'revenue_diff']])

Топ-3 сиквела, которые заработали больше предшественников:
                title  revenue_original  revenue_sequel  revenue_diff
28  Jurassic Park III       368780809.0      1513528810  1.144748e+09
26      Batman Begins       374218673.0      1004558444  6.303398e+08
11         Iron Man 2       623933331.0      1215439994  5.915067e+08


In [20]:
# 3. Какие режиссеры работали на других ролях
directors_df = crews[crews['job'] == 'Director']

non_director_roles_df = crews[(crews['name'].isin(directors_df['name'])) & (crews['job'] != 'Director')]

director_roles_with_titles_df = pd.merge(non_director_roles_df, movies, on='id')

director_roles_with_titles_df = director_roles_with_titles_df[['name', 'job', 'title']]
print("Режиссеры, работавшие на других ролях")
print(director_roles_with_titles_df)

Режиссеры, работавшие на других ролях
                   name                 job                      title
0         James Cameron              Writer                     Avatar
1      Robert Stromberg   Production Design                     Avatar
2     Christopher Nolan            Producer      The Dark Knight Rises
3        Andrew Stanton          Screenplay                John Carter
4         John Lasseter              Thanks                John Carter
...                 ...                 ...                        ...
1989        John Waters              Editor             Pink Flamingos
1990       Tadeo Garcia               Story             On The Downlow
1991      Shane Carruth      Cinematography                     Primer
1992   Robert Rodriguez            Producer                El Mariachi
1993        Scott Smith  Executive Producer  Signed, Sealed, Delivered

[1994 rows x 3 columns]


In [21]:
# Найдем, какие режиссеры выполняли больше всего ролей при съемке фильмов
diverse_directors = director_roles_with_titles_df.groupby('name')['job'].nunique().reset_index()
diverse_directors = diverse_directors.rename(columns={'job': 'number_of_jobs'}).sort_values(by='number_of_jobs', ascending=False)

print("Самые разносторонние режиссеры")
print(diverse_directors)


Самые разносторонние режиссеры
                            name  number_of_jobs
770             Robert Rodriguez              14
554                   Luc Besson               6
313                 George Lucas               5
14   Alejandro González Iñárritu               5
434               John Carpenter               5
..                           ...             ...
938               Wayne Thornley               1
937                 Wayne Kramer               1
936                  Wayne Beach               1
951            William A. Graham               1
935                Warren Beatty               1

[967 rows x 2 columns]
